### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [4]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (not= (get-date) "2016-12-29")
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (next-date))
;(update-eval-report (get-date))
(end-order)

*Interrupted*


interrupted: 

In [9]:
;(pprint/print-table (take 200 (deref portfolio-value)))
 (nth (deref order-record) 204)

[{:date "1980-12-15", :tot-value 1000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-16", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-17", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-18", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-19", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-22", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-23", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-24", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-26", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-29", :tot-value 1000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0} {:date "1980-12-30", :tot-value 1000.0, :

### Check portfolio record

In [16]:
;; view final portfolio
(view-portfolio)


| :asset | :price |  :aprc | :quantity | :tot-val |
|--------+--------+--------+-----------+----------|
|   cash |    N/A |    N/A |       N/A |   135759 |
|   AAPL | 116.73 | 312.11 |        81 |    25280 |
|      F |  14.87 | 605.11 |         0 |        0 |


nil

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [19]:
(eval-report)

nil

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)